In [2]:
# Initialize python libs & SQL creads
from datetime import datetime, timedelta, date
from dateutil.relativedelta import relativedelta
import pandas as pd
from pandasql import sqldf
import streamlit as st
import pull_nrg_data
import json
import http.client
import certifi
import ssl
import os
import timeit
import pickle
import altair as alt
import pytz
import numpy as np

In [37]:
t = datetime(2022,1,1).date()
default_pickle = {
    'accessToken':'',
    'current_data':'',
    'daily_outage_dfs':[0]*7,
    'monthly_outage_dfs':[0]*7,
    'alert_dates':{
        'Biomass & Other':t, 
        'Coal':t,
        'Dual Fuel':t,
        'Energy Storage':t,
        'Hydro':t,
        'Natural Gas':t,
        'Solar':t,
        'Wind':t,
        'Intertie':t
    }
}
with open('./default_pickle.pickle', 'wb') as handle:
    pickle.dump(default_pickle, handle, protocol=pickle.HIGHEST_PROTOCOL)
# Add default current values to dict
with open('./current_df.pickle', 'rb') as handle:
    current_df = pickle.load(handle)
default_pickle['current_data'] = current_df
# Add default monthly values to dict
with open('./monthly_df.pickle', 'rb') as handle:
    monthly = pickle.load(handle)
for i in range(7):
    default_pickle['monthly_outage_dfs'][i] = (datetime.today().date(),monthly)
# Add default daily values to dict
with open('./daily_df.pickle', 'rb') as handle:
    daily = pickle.load(handle)
for i in range(7):
    default_pickle['daily_outage_dfs'][i] = (datetime.today().date(),daily)
with open('./default_pickle.pickle', 'wb') as handle:
    pickle.dump(default_pickle, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [14]:
with open('./default_pickle.pickle', 'rb') as handle:
    default_pickle = pickle.load(handle)
default_pickle

{'accessToken': 'XR_YwicWmah8one19qhYiNdz4SX7-sXOQFOmRjIbddC2B0wB3_kmht-hKuu89cuvz0KqZUoK4mHkz2MuKzUZuDmhPFMp4iZeS5O3qBKSTdkLzGaCMYwddrfGciHbS8VzwdvXRua0uqRxw53uixF8wpfItSn77h6im0TNvX6ofu2nrJOjsVs_2_XrN-MznV0yVXMgq5yn2ieILdjT4880xL4O1NX25BEbJ_dJKcyZEax6Ur0R2HhQL0htKU7sRkCWFLg3quqnDBCjtoR8UNL1uPYwYNgUMjnN1IpgXy9D-YGhk-ZphUAMm-16jUzd8viZ2dqbvaqsMRfGVS74-I6eiig8ZPU',
 'current_data': (datetime.datetime(2022, 9, 9, 10, 5, 37, 436266, tzinfo=<DstTzInfo 'America/Edmonton' MDT-1 day, 18:00:00 DST>),
                timeStamp  value      fuelType
  0   2022-09-09 00:00:00   1240          Coal
  1   2022-09-09 00:05:00   1250          Coal
  2   2022-09-09 00:10:00   1245          Coal
  3   2022-09-09 00:15:00   1241          Coal
  4   2022-09-09 00:20:00   1240          Coal
  ..                  ...    ...           ...
  117 2022-09-09 09:45:00    -59  Saskatchewan
  118 2022-09-09 09:50:00    -59  Saskatchewan
  119 2022-09-09 09:55:00    -58  Saskatchewan
  120 2022-09-09 10:00:00    -59

In [15]:
default_pickle['daily_outage_dfs'][6]

(datetime.date(2022, 9, 9),
     timeStamp  value  fuelType
 0  2022-09-09    644  Intertie
 1  2022-09-10      0  Intertie
 2  2022-09-11      0  Intertie
 3  2022-09-12      0  Intertie
 4  2022-09-13      0  Intertie
 ..        ...    ...       ...
 85 2022-12-03      0      Wind
 86 2022-12-04      0      Wind
 87 2022-12-05      0      Wind
 88 2022-12-06      0      Wind
 89 2022-12-07      0      Wind
 
 [810 rows x 3 columns])